In [1]:
import numpy as np
import pandas as pd
import feather

In [2]:
raw_train = pd.read_csv('data/train.csv.zip' ,compression = 'zip')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
raw_train.head()

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0006,VAR_0007,VAR_0008,VAR_0009,...,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,0.0,0.0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
1,4,H,7,53,4448,B,1.0,0.0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
2,5,H,116,3,3464,C,0.0,0.0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
3,7,H,240,300,3200,C,0.0,0.0,False,False,...,98,98,998,999999998,998,998,9998,9998,RCC,0
4,8,R,72,261,2000,N,0.0,0.0,False,False,...,98,98,998,999999998,998,998,9998,9998,BRANCH,1


In [4]:
len(raw_train)

145231

In [5]:
raw_train.describe()

,ID,VAR_0002,VAR_0003,VAR_0004,VAR_0006,VAR_0007,VAR_0013,VAR_0014,VAR_0015,VAR_0016,...,VAR_1925,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,target
count,145231.000000,145231.000000,145231.000000,1.452310e+05,145175.000000,145175.00000,145175.000000,145175.000000,145175.000000,145175.000000,...,145231.000000,145231.000000,145231.000000,145231.000000,1.452310e+05,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000
mean,145124.981808,105.278040,88.492285,3.721946e+03,0.693907,0.51316,0.693907,0.494493,0.476763,1.447598,...,0.549965,86.664369,89.828762,914.417259,9.904497e+08,922.938505,936.487905,9942.162279,8849.839111,0.232547
std,83890.290627,143.351237,125.903323,1.228001e+04,1.604578,1.17531,1.604578,1.115085,1.000755,1.132784,...,6.361312,31.347700,26.928221,276.361514,9.725818e+07,212.087416,210.370833,743.153332,3175.797333,0.422457
min,2.000000,0.000000,0.000000,0.000000e+00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000
25%,72342.500000,24.000000,13.000000,1.790000e+03,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,...,0.000000,98.000000,98.000000,998.000000,1.000000e+09,998.000000,998.000000,9998.000000,9998.000000,0.000000
50%,145272.000000,60.000000,55.000000,2.500000e+03,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,...,0.000000,98.000000,98.000000,998.000000,1.000000e+09,998.000000,998.000000,9998.000000,9998.000000,0.000000
75%,217686.500000,132.000000,120.000000,3.600000e+03,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,0.000000,98.000000,98.000000,998.000000,1.000000e+09,998.000000,998.000000,9998.000000,9998.000000,0.000000
max,290463.000000,999.000000,999.000000,2.200000e+06,68.000000,66.00000,68.000000,67.000000,35.000000,39.000000,...,99.000000,99.000000,99.000000,999.000000,1.000000e+09,999.000000,999.000000,9999.000000,9999.000000,1.000000


In [6]:
const_cols = [col for col in raw_train.columns if raw_train[col].nunique() == 1]

In [7]:
clean_train = raw_train.drop(const_cols, axis = 1)

In [8]:
null_cols = [col for col in clean_train.columns if clean_train[col].isnull().sum()/len(clean_train) > 0]

In [9]:
object_cols = clean_train.select_dtypes(include = 'object').columns.tolist()

In [10]:
object_nulls = list(set(null_cols).intersection(set(object_cols)))

In [11]:
numeric_nulls = list(set(null_cols).intersection(set(clean_train.select_dtypes(include = 'number').columns.tolist())))

In [12]:
for col in numeric_nulls:
    clean_train[col+'_null_flag'] = np.where(clean_train[col].isnull(),1,0)
    clean_train[col].fillna(clean_train[col].mean(), inplace = True)

In [13]:
for col in object_nulls:
    clean_train[col+'_null_flag'] = np.where(clean_train[col].isnull(),1,0)
    clean_train[col].fillna(clean_train[col].value_counts().idxmax(), inplace = True)

In [14]:
for col in object_cols:
    print(col, clean_train[col].value_counts().idxmax())

VAR_0001 R
VAR_0005 B
VAR_0073 13MAR09:00:00:00
VAR_0075 22SEP10:00:00:00
VAR_0156 12DEC11:00:00:00
VAR_0157 27JUL12:00:00:00
VAR_0158 01FEB12:00:00:00
VAR_0159 04JUN12:00:00:00
VAR_0166 19DEC11:00:00:00
VAR_0167 03NOV11:00:00:00
VAR_0168 03NOV11:00:00:00
VAR_0169 19DEC11:00:00:00
VAR_0176 19DEC11:00:00:00
VAR_0177 03NOV11:00:00:00
VAR_0178 03NOV11:00:00:00
VAR_0179 04JUN12:00:00:00
VAR_0200 CHICAGO
VAR_0204 31JAN14:15:54:00
VAR_0214 HRE-Social Security Number-1747
VAR_0217 06DEC11:02:00:00
VAR_0226 False
VAR_0230 False
VAR_0232 True
VAR_0236 True
VAR_0237 CA
VAR_0274 CA
VAR_0283 S
VAR_0305 S
VAR_0325 S
VAR_0342 -1
VAR_0352 U
VAR_0353 U
VAR_0354 U
VAR_0404 -1
VAR_0466 -1
VAR_0467 -1
VAR_0493 -1
VAR_1934 IAPS


In [15]:
date_cols = ['VAR_0073', 'VAR_0075', 'VAR_0156', 'VAR_0157', 'VAR_0158', 'VAR_0159', 'VAR_0166', 'VAR_0167', 'VAR_0168', 'VAR_0169', 'VAR_0176'
            ,'VAR_0177', 'VAR_0178', 'VAR_0179', 'VAR_0204', 'VAR_0217']

In [16]:
for col in date_cols:
    clean_train[col] = pd.to_datetime(clean_train[col], format = '%d%b%y:%X')
    clean_train[col+'_weekday'] = clean_train[col].dt.weekday
    clean_train[col+"_day"] = clean_train[col].dt.day 
    clean_train[col+"_month"] = clean_train[col].dt.month
    clean_train[col+"_year"] = clean_train[col].dt.year

In [17]:
clean_train.drop(date_cols, axis = 1, inplace = True)

In [18]:
remaining_nulls = [col for col in clean_train.columns if clean_train[col].isnull().sum()>0]

In [19]:
remaining_nulls

['VAR_0207', 'VAR_0213', 'VAR_0840']

In [20]:
clean_train.drop(remaining_nulls, axis = 1, inplace = True)

In [21]:
raw_test = pd.read_csv('data/test.csv.zip' ,compression = 'zip')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
len(const_cols)

51

In [23]:
const_cols_test = [col for col in raw_test.columns if raw_test[col].nunique() == 1]

In [24]:
[col for col in const_cols_test if col not in const_cols]

['VAR_0526', 'VAR_0529']

In [25]:
raw_train['VAR_0526'].value_counts()

0.0    144313
1.0         1
Name: VAR_0526, dtype: int64

In [26]:
raw_test['VAR_0526'].value_counts()

0.0    144323
Name: VAR_0526, dtype: int64

In [27]:
raw_train['VAR_0529'].value_counts()

0.0    144313
1.0         1
Name: VAR_0529, dtype: int64

In [28]:
raw_test['VAR_0529'].value_counts()

0.0    144323
Name: VAR_0529, dtype: int64

In [29]:
clean_test = raw_test.drop(const_cols, axis = 1)

In [30]:
null_cols_test = [col for col in clean_test.columns if clean_test[col].isnull().sum()/len(clean_test) > 0]

In [31]:
[col for col in null_cols_test if col not in null_cols]

[]

In [32]:
object_cols_test = clean_test.select_dtypes(include = 'object').columns.tolist()

In [33]:
[col for col in object_cols_test if col not in object_cols]

[]

In [34]:
object_nulls_test = list(set(null_cols_test).intersection(set(object_cols_test)))

In [35]:
[col for col in object_nulls_test if col not in object_nulls]

[]

In [36]:
numeric_nulls_test = list(set(null_cols_test).intersection(set(clean_test.select_dtypes(include = 'number').columns.tolist())))

In [37]:
[col for col in numeric_nulls_test if col not in numeric_nulls]

[]

In [38]:
for col in numeric_nulls_test:
    clean_test[col+'_null_flag'] = np.where(clean_test[col].isnull(),1,0)
    clean_test[col].fillna(clean_test[col].mean(), inplace = True)

In [39]:
for col in object_nulls_test:
    clean_test[col+'_null_flag'] = np.where(clean_test[col].isnull(),1,0)
    clean_test[col].fillna(clean_test[col].value_counts().idxmax(), inplace = True)

In [40]:
for col in date_cols:
    clean_test[col] = pd.to_datetime(clean_test[col], format = '%d%b%y:%X')
    clean_test[col+'_weekday'] = clean_test[col].dt.weekday
    clean_test[col+"_day"] = clean_test[col].dt.day 
    clean_test[col+"_month"] = clean_test[col].dt.month
    clean_test[col+"_year"] = clean_test[col].dt.year

In [41]:
clean_test.drop(date_cols, axis = 1, inplace = True)

In [42]:
remaining_nulls_test = [col for col in clean_test.columns if clean_test[col].isnull().sum()>0]

In [43]:
remaining_nulls_test

['VAR_0207', 'VAR_0213', 'VAR_0840']

In [44]:
clean_test.drop(remaining_nulls, axis = 1, inplace = True)

In [46]:
[col for col in clean_test.columns.to_list() if col not in clean_train.columns.to_list()]

[]

In [47]:
feather.write_dataframe(clean_train, 'data/clean_train.feather')

In [48]:
feather.write_dataframe(clean_test, 'data/clean_test.feather')